In [130]:
import keyword
import torch
from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter('exp', flush_secs=1)
meta = []
while len(meta)<100:
    meta = meta+keyword.kwlist # get some strings
meta = meta[:100]

for i, v in enumerate(meta):
    meta[i] = v+str(i)

label_img = torch.rand(100, 3, 10, 32)
for i in range(100):
    label_img[i]*=i/100.0

writer.add_embedding(torch.randn(100, 5), metadata=meta, label_img=label_img, global_step=1)
# writer.add_embedding(torch.randn(100, 5), label_img=label_img)
# writer.add_embedding(torch.randn(100, 5), metadata=meta)

In [5]:
import torch

In [117]:
encoder_hidden_size = 3
decoder_hidden_size = 6

batch_size = 4
seq_len = 5

In [119]:
encoder_output = torch.randn(seq_len, batch_size, encoder_hidden_size)
encoder_output.shape

torch.Size([5, 4, 3])

In [120]:
# decoder_hidden = torch.tensor([[1, -1, 2, 1], [-1, -2, 1, -1]]).float()
decoder_hidden = torch.randn(batch_size, decoder_hidden_size)

print(decoder_hidden.shape)
print(decoder_hidden)

torch.Size([4, 6])
tensor([[-0.4113,  0.5070, -0.0952,  0.8271,  0.4745,  0.1033],
        [-0.4779, -2.2036,  0.3206,  1.3497,  0.4132,  1.2194],
        [ 0.6687, -0.5755, -0.2852, -2.9046,  0.3243, -0.3207],
        [-0.9867,  0.0064, -0.4692, -0.8371,  1.0053,  0.0793]])


In [125]:
class Attention(torch.nn.Module):
    
    def __init__(self, decoder_hidden_size, encoder_hidden_size):
        super(Attention, self).__init__()
        self.linear = torch.nn.Linear(
            decoder_hidden_size, encoder_hidden_size)
        
    def forward(self, decoder_hidden, encoder_output):
        h = self.linear(decoder_hidden)
        encoder_output = encoder_output.transpose(0, 1)
        a = torch.matmul(encoder_output, h.unsqueeze(-1)).squeeze()
        a = torch.softmax(a, dim=1)
        context = (a.unsqueeze(-1) * encoder_output).sum(1)
        return context
        
        
atten = Attention(
    decoder_hidden_size=decoder_hidden_size,
    encoder_hidden_size=encoder_hidden_size,
)
context = atten(decoder_hidden, encoder_output)
print(context.shape)
print(context)

torch.Size([4, 3])
tensor([[ 0.2489, -0.6795,  0.5418],
        [-1.0605,  0.3063,  1.2977],
        [-0.4571,  0.1155,  0.3059],
        [-1.6641, -1.0337,  0.4251]], grad_fn=<SumBackward1>)


In [111]:
def forward(output, hidden):
    output = output.transpose(0, 1)
    a = torch.matmul(output, hidden.unsqueeze(-1)).squeeze()
    a = torch.softmax(a, dim=1)
    context = (a.unsqueeze(-1) * output).sum(1)
    
    
contex = forward(output, hidden)

print(context.shape)
print(context)

torch.Size([2, 4])
tensor([[ 0.6601, -1.0603, -0.0714,  0.7535],
        [-0.0426, -0.5320,  0.3525, -0.6404]])
